# Revised case normalization for Hirslanden Beau Site

This jupyter notebook is used to normalize the revised case from DtoD.

Before runing the notebook, raw_data folder need to added to the root directory

The raw data folder can be find here: https://aimedic.sharepoint.com/:f:/s/dev/Ejx_A1dg8gtPumFknOWOh0oBi6ofx9hctYiq3c-0gH9vYA?e=UmcgrS

Normalization:

-  Convert the column names to the name used in the Database
-  Delete cases which is empty in the follow columns VALIDATION_COLS: 'case_id', 'patient_id', 'gender', 'age_years', duration_of_stay',  'pccl',  'drg'
- choose neccessary columns COLS_TO_SELECT: case_id, patient_id, gender, age_years, duration_of_stay, pccl, drg, pd, bfs_code, added_icds, removed_icds, added_chops, removed_chops
- still need to do (TODO):    
    -  Check CHOP upper/lowercase
    -  Check whether the PD changed. If it did, new and old PD are stored together with added and removed ICDs, respectively
    -  Pad case IDs with 0s
    -  Write function to validate cases


In [52]:
import pandas as pd
import os
from dataclasses import dataclass, field
import sys
sys.path.insert(0, '/home/jovyan/work')
sys.path.insert(1, '/home/jovyan/work/src/service')

from service import bfs_cases_db_service as bfs_db

from py.global_configs import *

In [53]:
FILES_TO_ANALYZE = {
    'Änderungen Beau Site 2017': FileInfo(
         os.path.join(ROOT_DIR, 'raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'),
         ['SA', 'Hirslanden Beau Site', 'Hirslanden Linde', 'Hirslanden Linde', 'SA', 'Hirslanden Beau Site'],
         ['2017', '2017', '2017', '2018', '2018', '2018'],
         ['Änderungen_SA_2017', 'Änderungen Beau Site 2017', 'Änderungen_LI_2017', 'Änderungen_LI_2018', 'Änderungen _SA_2018', 'Änderungen Beau Site_ 2018']),


In [54]:
def normalize(fi: FileInfo,
              *, 
              columns_mapper: dict = COLUMNS_TO_RENAME,
              columns_to_cast: dict = COLUMNS_TO_CAST,
              columns_to_lstrip: set = COLUMNS_TO_LSTRIP
              ) -> pd.DataFrame:
    """
    """
    
    revised_cases = list()
    for i ,sheet in enumerate(fi.sheets):
        file_path = fi.path
        file_name = fi.sheets[i]
        # Read the Excel file and sheet. Cast all columns to strings, so we can format / cast the columns ourselves later on.
        # `string[pyarrow]` is an efficient way of storing strings in a DataFrame
        df = pd.read_excel(file_path, sheet_name=file_name, dtype='string[pyarrow]')
        # Convert all column names to lower-case, so we don't have to deal with columns named `HD Alt` vs `HD alt`
        df.columns = [c.lower() for c in df.columns]
        # Renaming columns that don't exist is a no-op. Make sure that all names actually exist
        assert(len(set(columns_mapper.keys()).difference(df.columns)) == 0)
        df.rename(columns=columns_mapper, inplace=True)

        assert(len(set(COLS_TO_SELECT).difference(df.columns)) == 0)
        df = df[COLS_TO_SELECT]
        n_all_rows = df.shape[0]
        print(f'Read {n_all_rows} cases for {fi.hospital_name_db[i]} {fi.year[i]}')

        # Remove rows where any value is NaN
        assert(len(set(VALIDATION_COLS).difference(df.columns)) == 0)
        df.dropna(subset=VALIDATION_COLS, inplace=True)
        n_valid_rows = df.shape[0]
        if n_valid_rows < n_all_rows:
            print(f'{n_all_rows - n_valid_rows}/{n_all_rows} rows were deleted because contained NaNs')

        # Cast columns to correct data type (according to DB)
        assert(len(set(columns_to_cast.keys()).difference(df.columns)) == 0)
        for col_name, col_type in columns_to_cast.items():
            df[col_name] = df[col_name].astype(col_type)  

        print(f'TYPES:\n{df.dtypes}')

        # Fix format of some columns
        lstrip_fun = lambda x: x.lstrip("'")
        for col_name in columns_to_lstrip:
            df[col_name] = df[col_name].apply(lstrip_fun)
            
        # add hospital name and year as two columns
        hospital_name_db = fi.hospital_name_db[i]
        year = fi.year[i]
        df['hospital_name_db'] = hospital_name_db
        df['year'] = year
        revised_cases.append(df)
    revised_cases_df = pd.concat(revised_cases)
    
    return revised_cases_df
        

    # TODO Check CHOP upper/lowercase
    # TODO Check whether the PD changed. If it did, new and old PD are stored together with added and removed ICDs, respectively
    # TODO Pad case IDs with 0s
    # TODO Write function to validate cases


## Revised cases from excel 'HI-Bern_Salem_Beau Site_Linde.xlsx'


In [55]:
# All excel files
FILES_TO_ANALYZE.keys()


dict_keys(['HI-Bern_Salem_Beau Site_Linde.xlsx', 'HI-Zurich.xlsx', 'HI_Aarau_Birshof_ST. Anna.xlsx', 'KSSG_2021.xlsx', 'Linth_Toggenburg_SRRWS_2019.xlsx', 'USZ_2018-2019_20200730.xlsx', 'Winterthur.xlsx'])

In [56]:
file_hi_bern = FILES_TO_ANALYZE['HI-Bern_Salem_Beau Site_Linde.xlsx']
file_hi_bern

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db=['SA', 'Hirslanden Beau Site', 'Hirslanden Linde', 'Hirslanden Linde', 'SA', 'Hirslanden Beau Site'], year=['2017', '2017', '2017', '2018', '2018', '2018'], sheets=['Änderungen_SA_2017', 'Änderungen Beau Site 2017', 'Änderungen_LI_2017', 'Änderungen_LI_2018', 'Änderungen _SA_2018', 'Änderungen Beau Site_ 2018'])

In [57]:
file_hi_bern_norm = normalize(file_hi_bern)
file_hi_bern_norm

Read 57 cases for SA 2017
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
pd                  string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object
Read 47 cases for Hirslanden Beau Site 2017
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
pd                  string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object
Read 18 cases for Hirslanden Linde 2017
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops,hospital_name_db,year
0,41194601,8A6B7DE0296BEFB8,W,73,10,3,I09C,M4806,M200,N183,<NA>,948x40::20170128,948X40::20170128,SA,2017
1,41248206,8DAB6117ADBB2AFE,M,84,8,3,I43B,M179,M200,I420,<NA>,99b812::20170212,99B812::20170212,SA,2017
2,41302785,5865672002F07F3F,W,97,13,3,F62C,I5001,M100,"E876,A099",<NA>,<NA>,<NA>,SA,2017
3,41304308,0CD7621F494A9140,M,36,3,3,L63D,N390,M100,"G8202,Q051",<NA>,<NA>,399521::20170304,SA,2017
4,41237114,376800437C6F8D5B,M,78,7,3,I09D,M9963,M200,"T842,F101",<NA>,<NA>,<NA>,SA,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,0041830088,0022197396,M,63,11,3,G18B,C187,M200,K566,<NA>,<NA>,<NA>,Hirslanden Beau Site,2018
22,0041861755,0005180098,W,86,1,2,K60F,E1160,M100,G2090,<NA>,<NA>,<NA>,Hirslanden Beau Site,2018
23,0041863191,0005147943,W,85,3,3,H61B,D376,M200,"N183,K767,D684",N185,<NA>,<NA>,Hirslanden Beau Site,2018
24,0041869562,0022192074,W,95,9,3,F62C,I5001,M100,<NA>,I5001,<NA>,<NA>,Hirslanden Beau Site,2018


In [62]:
bfs_cases_db_service.get_bfs_cases_by_ids(['0041650906'])

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl


In [ ]:
# connect to the database

# collect all the case id, make a query and return all data
# find the one did not match
# modify the one did not match, i.e. add 000 in front, 




# search with patient_id , together clinic, year, age ?????





In [ ]:
## Reivsed case from excel file 'HI-Zurich.xlsx'

In [ ]:
# All excel files
FILES_TO_ANALYZE.keys()


In [ ]:
file_hi_zurich = FILES_TO_ANALYZE['HI-Zurich.xlsx']
file_hi_zurich

In [ ]:
file_hi_zurich_norm = normalize(file_hi_zurich)
file_hi_zurich_norm

In [ ]:
# need to take care the 'n.ü.' in the duration of stay

## Revised case from excel 'HI_Aarau_Birshof_ST. Anna.xlsx'

In [ ]:
FILES_TO_ANALYZE.keys()

In [ ]:
file_hi_aurau = FILES_TO_ANALYZE['HI_Aarau_Birshof_ST. Anna.xlsx']
file_hi_aurau

In [ ]:
file_hi_aurau_norm = normalize(file_hi_aurau)

In [ ]:
file_hi_aurau_norm

## Revised case from excel file 'KSSG_2021.xlsx'

In [ ]:
FILES_TO_ANALYZE.keys()

In [ ]:
file_kssg_2021 = FILES_TO_ANALYZE['KSSG_2021.xlsx']
file_kssg_2021

In [ ]:
kssg_2021_norm = normalize(file_kssg_2021)

In [ ]:
kssg_2021_norm

## Revised case from excel file 'Linth_Toggenburg_SRRWS_2019.xlsx'

In [ ]:
FILES_TO_ANALYZE.keys()

In [ ]:
file_linth = FILES_TO_ANALYZE['Linth_Toggenburg_SRRWS_2019.xlsx']
file_linth

In [ ]:
file_linth_norm = normalize(file_linth)

In [ ]:
file_linth_norm

## Revised cases from excel file 'USZ_2018-2019_20200730.xlsx'

In [ ]:
FILES_TO_ANALYZE.keys()

In [ ]:
file_usz = FILES_TO_ANALYZE['USZ_2018-2019_20200730.xlsx']
file_usz

In [ ]:
file_usz_norm = normalize(file_usz)

In [ ]:
file_usz_norm

## Revised cases from excel file 'Winterthur.xlsx'

Note:  Winterthur 2018 and Winterthur 2019 do not have AdmNo. i.e. case_id. There are excluded in the global_config.py


In [ ]:
#TODO: need to check what to do with winterthur 2018, 2019


In [ ]:
FILES_TO_ANALYZE.keys()

In [ ]:
file_winterthur = FILES_TO_ANALYZE['Winterthur.xlsx']
file_winterthur

In [ ]:
COLUMNS_TO_RENAME

In [ ]:
file_wintertur_2018 = pd.read_excel(file_winterthur.path, sheet_name=file_winterthur.sheets[1], dtype='string[pyarrow]')
col_names = [col.lower() for col in file_wintertur_2018.columns]

In [ ]:
col_names

In [ ]:
file_winterthur_norm = normalize(file_winterthur)
file_winterthur_norm